In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2025-10-03 00:49:56--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.1’

book-crossings.zip. 100%[===================>]  24.88M  --.-KB/s    in 0.06s   

2025-10-03 00:49:56 (386 MB/s) - ‘book-crossings.zip.1’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
# add your code here - consider creating a new cell for each section of code
# Filter ineligible users and books
# Thanks https://forum.freecodecamp.org/t/book-recommendation-knn-system/412976/9 for filtering method
users = df_ratings['user'].value_counts()
books = df_ratings['isbn'].value_counts()

df_ratings = df_ratings[df_ratings['user'].isin(users[users >= 200].index)]
df_ratings = df_ratings[df_ratings['isbn'].isin(books[books >= 100].index)]

# Merge books and ratings into one dataframe
books_and_ratings = pd.merge(df_books, df_ratings, on='isbn')

# A pivot table is needed to fill non numeric values, thanks https://medium.com/@madhanr.hi/k-nearest-neighbors-6753c052eb06
pivot = books_and_ratings.pivot_table(index='title', columns='user', values='rating')
pivot.fillna(0, inplace=True)

# Thanks https://scikit-learn.org/stable/modules/neighbors.html for NearestNeighbors reference
nbrs = NearestNeighbors(metric='cosine', algorithm='brute').fit(pivot.values)
distances, indices = nbrs.kneighbors(pivot.values, n_neighbors=6)

# Get titles in dataset
titles = pivot.index.to_list()

In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  # Get index of book
  try:
    index = titles.index(book)
  except ValueError:
    return []

  # Add title to beginning of output
  recommended_books = []
  recommended_books.append(book)
  recommended_books.append([])

  # Add corresponding bookd and distances, skipping current book
  book_titles = indices[index]
  j = 0
  for i in distances[index]:
    if titles[book_titles[j]] != book:
      # Thanks https://stackoverflow.com/a/11389998 for item method
      recommended_books[1].append([titles[book_titles[j]], i.item()])
    j += 1

  # Need to trim book since this pulls in one too many, reverse list
  recommended_books[1] = recommended_books[1][1::]
  recommended_books[1] = recommended_books[1][::-1]

  return recommended_books

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016210794448853], ['The Weight of Water', 0.7708583474159241], ['The Surgeon', 0.7699410915374756], ['I Know This Much Is True', 0.7677075266838074]]]
You passed the challenge! 🎉🎉🎉🎉🎉
